In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [2]:
class Mymodel(keras.Model):
    def __init__(self):
        super(Mymodel, self).__init__()

        self.CNN = keras.Sequential([
            ## build the CNN layers
            keras.layers.Conv2D(filters=32, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Conv2D(filters=64, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Conv2D(filters=96, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Conv2D(filters=128, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(10, activation='softmax')
        ])

    def call(self, x):
        return self.CNN(x)

In [3]:
def load_dataset(path):
    X = []
    for i in range(1,10):
        data = np.load(path+'dataX'+ str(i) +'.npz')
        X.append(data['arr'])
    Y = np.load(path+'dataY.npz')
    Y = np.array(Y['arr'])
    X1 = X[0]
    for i in range(1,len(X)):
        X1 = np.concatenate((X1,X[i]))
    return X1,Y

In [4]:
X,Y=load_dataset('data/')
print(X.shape, Y.shape)

(8732, 40, 173) (8732, 10)


In [5]:
## shuffle the datasets, makes train better 
idx  = np.arange(8732)
np.random.shuffle(idx)
## [0,6986） train  80% ; [6986，8732) test 20%
Xs,Ys = X[idx],Y[idx]
trainX, testX = Xs[0:6986],Xs[6986:8732]
trainY, testY = Ys[0:6986],Ys[6986:8732]
print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(6986, 40, 173) (6986, 10)
(1746, 40, 173) (1746, 10)


In [6]:
# reshape the train data
trainX = trainX.reshape((-1,40,173,1))
testX = testX.reshape((-1,40,173,1))
print(trainX.shape, testX.shape)

(6986, 40, 173, 1) (1746, 40, 173, 1)


In [7]:
## build the model
model=Mymodel()
model.compile(optimizer=keras.optimizers.Adam(),
              loss = keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])


In [8]:
## fit the model
model.fit(trainX, trainY, batch_size=64, epochs=10)


Train on 6986 samples
Epoch 1/10
6986/6986 [==============================] - 53s 8ms/sample - loss: 1.9887 - accuracy: 0.2322
Epoch 2/10
6986/6986 [==============================] - 63s 9ms/sample - loss: 1.5834 - accuracy: 0.4014
Epoch 3/10
6986/6986 [==============================] - 53s 8ms/sample - loss: 1.3584 - accuracy: 0.4948
Epoch 4/10
6986/6986 [==============================] - 53s 8ms/sample - loss: 1.2054 - accuracy: 0.5608
Epoch 5/10
6986/6986 [==============================] - 53s 8ms/sample - loss: 1.0461 - accuracy: 0.6188
Epoch 6/10
6986/6986 [==============================] - 53s 8ms/sample - loss: 0.9508 - accuracy: 0.6691
Epoch 7/10
6986/6986 [==============================] - 53s 8ms/sample - loss: 0.8236 - accuracy: 0.7134
Epoch 8/10
6986/6986 [==============================] - 53s 8ms/sample - loss: 0.7481 - accuracy: 0.7413
Epoch 9/10
6986/6986 [==============================] - 53s 8ms/sample - loss: 0.6704 - accuracy: 0.7627
Epoch 10/10
6986/6986 [==========

In [9]:
## envaluate the model, calculate the accuracy in test data
model.evaluate(testX,testY)

1746/1746 [==============================] - 5s 3ms/sample - loss: 0.6877 - accuracy: 0.76400s - loss: 0.6879 - accu


[0.687723212294016, 0.76403207]

In [10]:
## predict Y with testX
predY = model.predict(testX)

In [11]:
# show the model 
model.summary()

Model: "mymodel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  514026    
Total params: 514,026
Trainable params: 514,026
Non-trainable params: 0
_________________________________________________________________


In [12]:
# save the model weights
model.save_weights('weights/weights')